In [1]:
import pandas as pd
import numpy
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error,r2_score
import yfinance as yf

#Used Models
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor

In [2]:
symbol='INFY.NS'
ticker = yf.Ticker(symbol)
data=ticker.history(period='max',interval='1m')
data.reset_index(inplace=True)

In [3]:
data.head()

,Datetime,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2024-04-04 09:15:00+05:30,1491.449951,1491.449951,1487.500000,1487.500000,0,0.0,0.0
1,2024-04-04 09:16:00+05:30,1488.449951,1489.800049,1488.150024,1488.400024,83875,0.0,0.0
2,2024-04-04 09:17:00+05:30,1488.449951,1488.699951,1486.000000,1487.949951,68944,0.0,0.0
3,2024-04-04 09:18:00+05:30,1487.900024,1487.949951,1485.849976,1486.150024,51406,0.0,0.0
4,2024-04-04 09:19:00+05:30,1486.150024,1486.599976,1485.199951,1486.000000,20556,0.0,0.0


In [4]:
data.tail()

,Datetime,Open,High,Low,Close,Volume,Dividends,Stock Splits
1868,2024-04-10 15:25:00+05:30,1506.599976,1506.800049,1505.250000,1505.750000,28855,0.0,0.0
1869,2024-04-10 15:26:00+05:30,1505.750000,1505.750000,1505.500000,1505.750000,37680,0.0,0.0
1870,2024-04-10 15:27:00+05:30,1505.750000,1506.800049,1505.650024,1506.800049,35871,0.0,0.0
1871,2024-04-10 15:28:00+05:30,1506.500000,1507.000000,1506.000000,1506.699951,22478,0.0,0.0
1872,2024-04-10 15:29:00+05:30,1506.800049,1506.800049,1506.000000,1506.800049,6742466,0.0,0.0


In [5]:
# data = pd.read_csv('/content/Nifty50_shortTerm_5m_New.csv')

print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1873 entries, 0 to 1872
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype                       
---  ------        --------------  -----                       
 0   Datetime      1873 non-null   datetime64[ns, Asia/Kolkata]
 1   Open          1873 non-null   float64                     
 2   High          1873 non-null   float64                     
 3   Low           1873 non-null   float64                     
 4   Close         1873 non-null   float64                     
 5   Volume        1873 non-null   int64                       
 6   Dividends     1873 non-null   float64                     
 7   Stock Splits  1873 non-null   float64                     
dtypes: datetime64[ns, Asia/Kolkata](1), float64(6), int64(1)
memory usage: 117.2 KB
None


In [6]:
print(data.describe())


              Open         High          Low        Close        Volume  \
count  1873.000000  1873.000000  1873.000000  1873.000000  1.873000e+03   
mean   1489.167698  1489.641429  1488.617246  1489.160142  2.417724e+04   
std      10.791055    10.803558    10.774792    10.800830  1.571996e+05   
min    1469.050049  1469.800049  1468.099976  1468.800049  0.000000e+00   
25%    1480.000000  1480.349976  1479.599976  1480.000000  6.702000e+03   
50%    1485.000000  1485.699951  1484.300049  1484.800049  1.281400e+04   
75%    1498.050049  1498.550049  1497.550049  1498.050049  2.541100e+04   
max    1513.699951  1513.800049  1512.650024  1513.750000  6.742466e+06   

       Dividends  Stock Splits  
count     1873.0        1873.0  
mean         0.0           0.0  
std          0.0           0.0  
min          0.0           0.0  
25%          0.0           0.0  
50%          0.0           0.0  
75%          0.0           0.0  
max          0.0           0.0  


In [7]:
data.drop_duplicates()

,Datetime,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2024-04-04 09:15:00+05:30,1491.449951,1491.449951,1487.500000,1487.500000,0,0.0,0.0
1,2024-04-04 09:16:00+05:30,1488.449951,1489.800049,1488.150024,1488.400024,83875,0.0,0.0
2,2024-04-04 09:17:00+05:30,1488.449951,1488.699951,1486.000000,1487.949951,68944,0.0,0.0
3,2024-04-04 09:18:00+05:30,1487.900024,1487.949951,1485.849976,1486.150024,51406,0.0,0.0
4,2024-04-04 09:19:00+05:30,1486.150024,1486.599976,1485.199951,1486.000000,20556,0.0,0.0
...,...,...,...,...,...,...,...,...
1868,2024-04-10 15:25:00+05:30,1506.599976,1506.800049,1505.250000,1505.750000,28855,0.0,0.0
1869,2024-04-10 15:26:00+05:30,1505.750000,1505.750000,1505.500000,1505.750000,37680,0.0,0.0
1870,2024-04-10 15:27:00+05:30,1505.750000,1506.800049,1505.650024,1506.800049,35871,0.0,0.0
1871,2024-04-10 15:28:00+05:30,1506.500000,1507.000000,1506.000000,1506.699951,22478,0.0,0.0


In [8]:
data.isnull().sum()

Datetime        0
Open            0
High            0
Low             0
Close           0
Volume          0
Dividends       0
Stock Splits    0
dtype: int64

In [9]:
data.drop(['Datetime','Volume','Dividends','Stock Splits'],axis=1,inplace=True)
data.head(10)



,Open,High,Low,Close
0,1491.449951,1491.449951,1487.500000,1487.500000
1,1488.449951,1489.800049,1488.150024,1488.400024
2,1488.449951,1488.699951,1486.000000,1487.949951
3,1487.900024,1487.949951,1485.849976,1486.150024
4,1486.150024,1486.599976,1485.199951,1486.000000
5,1486.349976,1486.750000,1485.250000,1485.699951
6,1485.050049,1487.000000,1483.750000,1485.550049
7,1485.750000,1487.699951,1485.099976,1487.699951
8,1487.250000,1488.000000,1485.250000,1486.599976
9,1486.500000,1487.000000,1485.800049,1486.699951


In [10]:
data['shift_close'] = data['Close'].shift(-1)
data.dropna(inplace=True)

In [11]:
data

,Open,High,Low,Close,shift_close
0,1491.449951,1491.449951,1487.500000,1487.500000,1488.400024
1,1488.449951,1489.800049,1488.150024,1488.400024,1487.949951
2,1488.449951,1488.699951,1486.000000,1487.949951,1486.150024
3,1487.900024,1487.949951,1485.849976,1486.150024,1486.000000
4,1486.150024,1486.599976,1485.199951,1486.000000,1485.699951
...,...,...,...,...,...
1867,1506.900024,1507.000000,1506.800049,1506.900024,1505.750000
1868,1506.599976,1506.800049,1505.250000,1505.750000,1505.750000
1869,1505.750000,1505.750000,1505.500000,1505.750000,1506.800049
1870,1505.750000,1506.800049,1505.650024,1506.800049,1506.699951


In [12]:
data.tail()

,Open,High,Low,Close,shift_close
1867,1506.900024,1507.000000,1506.800049,1506.900024,1505.750000
1868,1506.599976,1506.800049,1505.250000,1505.750000,1505.750000
1869,1505.750000,1505.750000,1505.500000,1505.750000,1506.800049
1870,1505.750000,1506.800049,1505.650024,1506.800049,1506.699951
1871,1506.500000,1507.000000,1506.000000,1506.699951,1506.800049


In [13]:
X=data.drop('shift_close',axis=1)
y = data['shift_close']

In [14]:
xy=pd.concat([X,y],axis=1)
xy.tail()

,Open,High,Low,Close,shift_close
1867,1506.900024,1507.000000,1506.800049,1506.900024,1505.750000
1868,1506.599976,1506.800049,1505.250000,1505.750000,1505.750000
1869,1505.750000,1505.750000,1505.500000,1505.750000,1506.800049
1870,1505.750000,1506.800049,1505.650024,1506.800049,1506.699951
1871,1506.500000,1507.000000,1506.000000,1506.699951,1506.800049


In [15]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [16]:

my_model = XGBRegressor()
my_model.fit(X_train, y_train)

predictions = my_model.predict(X_test)
print("Mean Absolute Error: " + str(mean_absolute_error(predictions, y_test)))

print("R2 Score" + str(r2_score(y_test,predictions)))


print("Accuracy Score : "+str(my_model.score(X_test,y_test)))

predictions


Mean Absolute Error: 0.7271917317708333
R2 Score0.9880071352240202
Accuracy Score : 0.9880071352240202


array([1492.3657, 1486.5095, 1507.8163, 1476.036 , 1504.1599, 1493.9849,
       1497.0397, 1482.7017, 1497.3978, 1475.1456, 1506.801 , 1505.1699,
       1507.2179, 1488.3042, 1480.1595, 1505.494 , 1500.0275, 1497.922 ,
       1506.9364, 1479.9125, 1482.4302, 1478.7008, 1494.2675, 1498.8851,
       1496.2119, 1479.1392, 1483.0775, 1490.996 , 1504.191 , 1485.5616,
       1499.4615, 1496.7422, 1476.578 , 1485.5916, 1482.0444, 1477.5354,
       1504.1699, 1480.3053, 1476.8593, 1481.8851, 1506.3983, 1477.7496,
       1478.4852, 1482.01  , 1510.5718, 1481.7783, 1480.4686, 1486.6023,
       1481.8811, 1479.5122, 1505.0208, 1506.4884, 1478.7332, 1499.6932,
       1477.9762, 1498.6759, 1482.4991, 1478.0516, 1475.3164, 1478.9054,
       1508.4626, 1477.484 , 1482.9778, 1483.193 , 1479.2893, 1481.7106,
       1484.0054, 1482.7764, 1491.7532, 1499.0312, 1474.9915, 1475.4843,
       1482.868 , 1489.0804, 1507.6022, 1481.3357, 1505.1047, 1496.5411,
       1479.0182, 1481.7147, 1488.4292, 1478.3976, 

In [17]:
RandomeForestModel = RandomForestRegressor()
RandomeForestModel.fit(X_train, y_train)


predictions = RandomeForestModel.predict(X_test)
print("Mean Absolute Error: " + str(mean_absolute_error(predictions, y_test)))

print("R2 Score" + str(r2_score(y_test,predictions)))


print("Accuracy Score : "+str(RandomeForestModel.score(X_test,y_test)))

predictions

Mean Absolute Error: 0.6770767561848983
R2 Score0.9887120731429849
Accuracy Score : 0.9887120731429849


array([1492.06199585, 1487.21650635, 1507.81549194, 1475.55401123,
       1504.22347168, 1494.48699341, 1496.97798462, 1482.58448242,
       1497.66700439, 1475.22203125, 1506.66599854, 1505.09252686,
       1507.38001587, 1488.89850586, 1480.27502197, 1505.37701782,
       1500.36097534, 1497.49199829, 1507.22500977, 1479.97595337,
       1482.34299561, 1478.64598511, 1494.29399414, 1498.82251709,
       1496.45547852, 1478.91297729, 1483.11249634, 1491.78402466,
       1504.39000244, 1485.59400757, 1499.56850952, 1496.34797607,
       1476.42200562, 1486.20399902, 1482.15699829, 1477.49549438,
       1504.10600586, 1479.83202759, 1477.0555127 , 1481.76099854,
       1506.33750732, 1478.25901367, 1478.48748901, 1482.64649902,
       1510.13298706, 1481.78099976, 1480.24900879, 1485.61200195,
       1481.92951416, 1479.38449829, 1505.2875    , 1506.53799927,
       1478.75751099, 1499.20749878, 1478.18099854, 1498.73752686,
       1482.49249878, 1478.75899902, 1475.46750732, 1478.88350

In [18]:
DesitionTreeModel = DecisionTreeRegressor()
DesitionTreeModel.fit(X_train, y_train)


predictions = DesitionTreeModel.predict(X_test)
print("Mean Absolute Error: " + str(mean_absolute_error(predictions, y_test)))

print("R2 Score" + str(r2_score(y_test,predictions)))


print("Accuracy Score : "+str(DesitionTreeModel.score(X_test,y_test)))

predictions

Mean Absolute Error: 0.8579313151041666
R2 Score0.9845495860053739
Accuracy Score : 0.9845495860053739


array([1491.30004883, 1487.15002441, 1507.80004883, 1477.05004883,
       1504.19995117, 1495.65002441, 1496.44995117, 1482.59997559,
       1497.59997559, 1475.05004883, 1506.94995117, 1504.80004883,
       1508.25      , 1489.        , 1480.30004883, 1505.30004883,
       1500.69995117, 1497.5       , 1508.94995117, 1479.97497559,
       1482.        , 1477.94995117, 1495.59997559, 1499.40002441,
       1496.30004883, 1477.94995117, 1482.5       , 1491.05004883,
       1504.15002441, 1485.84997559, 1500.05004883, 1495.44995117,
       1476.5       , 1486.69995117, 1482.05004883, 1477.59997559,
       1504.05004883, 1479.55004883, 1477.25      , 1481.80004883,
       1506.75      , 1477.80004883, 1479.25      , 1482.25      ,
       1509.59997559, 1481.34997559, 1480.40002441, 1482.25      ,
       1481.80004883, 1479.59997559, 1504.44995117, 1506.69995117,
       1478.65002441, 1498.59997559, 1478.69995117, 1498.90002441,
       1482.75      , 1478.25      , 1475.90002441, 1478.40002

In [19]:

import pickle
pickle.dump(my_model,open('/home/rahul/Desktop/Rahul/DE SEM 6/stock-prediction-bot/technical_analysis/technical_analysis_model/model_pkl_files/XGBoost.pkl','wb'))
pickle.dump(RandomeForestModel,open('/home/rahul/Desktop/Rahul/DE SEM 6/stock-prediction-bot/technical_analysis/technical_analysis_model/model_pkl_files/RandomForestRegressor.pkl','wb'))
pickle.dump(DesitionTreeModel,open('/home/rahul/Desktop/Rahul/DE SEM 6/stock-prediction-bot/technical_analysis/technical_analysis_model/model_pkl_files/DesicionTreeModel.pkl','wb'))

# pickle.dump(my_model,open('/content/drive/MyDrive/AI COLLAGE/5m_Nifty_RandomForestRegressor.pkl','wb'))


import yfinance as yf
import datetime as dt
import pickle
from sklearn.metrics import mean_absolute_error,r2_score,accuracy


# Load the model from the pickle file
with open('/home/rahul/Desktop/Rahul/DE SEM 6/stock-prediction-bot/technical_analysis/technical_analysis_model/model_pkl_files/RandomForestRegressor.pkl', 'rb') as f:
    Rand = pickle.load(f)


with open('/home/rahul/Desktop/Rahul/DE SEM 6/stock-prediction-bot/technical_analysis/technical_analysis_model/model_pkl_files/DesicionTreeModel.pkl', 'rb') as f:
    Decision = pickle.load(f)


with open('/home/rahul/Desktop/Rahul/DE SEM 6/stock-prediction-bot/technical_analysis/technical_analysis_model/model_pkl_files/XGBoost.pkl', 'rb') as f:
    XgBoostModel = pickle.load(f)



while True:
    ticker = yf.Ticker('^NSEI')
    start=dt.datetime.now().strftime('%Y-%m-%d')
    end=(dt.datetime.now() + dt.timedelta(days=1)).strftime('%Y-%m-%d')

    data = ticker.history(interval='5m',start=start,end=end)
    data.reset_index(inplace=True)
    # print(data)
    data.columns = data.columns.str.strip()  # Remove extra spaces from column names

    data.drop(['Datetime', 'Volume', 'Dividends', 'Stock Splits'], inplace=True, axis=1)
    y = data['Close'].copy()
    data.drop(['Close'], inplace=True, axis=1)

    print("Random Forest Regressor")
    print("---------------------------\n")
    Rand_predic=Rand.predict(data)
    last_pred=Rand_predic[-1]
    print(f'Last Prediction ----------> :{ last_pred}')
    print(f'mae : {mean_absolute_error(y,Rand_predic)}')
    # print(f"Rand Accuracy Score :{}")
    print(f'r2 : {r2_score(y,Rand_predic)}')
    print("---------------------------")
    print("\n\n\n")

    print("Desicion Tree Regressor")
    print("---------------------------\n")
    DecisionPred=Decision.predict(data)
    last_pred=DecisionPred[-1]
    mae=mean_absolute_error(y,DecisionPred)
    # last_pred=last_pred+mae
    print(f'Last Prediction ----------> :{ last_pred}')
    print(f'mae : {mae}')
    print(f'r2 : {r2_score(y,DecisionPred)}')
    print("---------------------------")
    print("\n\n\n")

    print("XgBoostModel ")
    print("---------------------------\n")
    XgBoostPred=XgBoostModel.predict(data)
    last_pred=XgBoostPred[-1]
    print(f'Last Prediction ----------> :{ last_pred}')
    print(f'mae : {mean_absolute_error(y,XgBoostPred)}')
    print(f'r2 : {r2_score(y,XgBoostPred)}')
    print("---------------------------")
    print("\n\n\n")
    # data